In [1]:
from pathlib import Path
import os
import pandas as pd
import numpy as np

In [2]:
BASE_PATH = Path.cwd()
DATA_PATH = BASE_PATH / "data"
MODEL_PATH = BASE_PATH / "model"
print(f"BASE_PATH: {BASE_PATH}")
print(f"DATA_PATH: {DATA_PATH}")
print(f"MODEL_PATH: {MODEL_PATH}")

BASE_PATH: /home/masa1357/Dockerdata/kaggle/Kaggle_Hull-Tactical---Market-Prediction
DATA_PATH: /home/masa1357/Dockerdata/kaggle/Kaggle_Hull-Tactical---Market-Prediction/data
MODEL_PATH: /home/masa1357/Dockerdata/kaggle/Kaggle_Hull-Tactical---Market-Prediction/model


In [3]:
train = pd.read_csv(DATA_PATH / "train.csv")
train.describe()

,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
count,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,...,7984.000000,7984.000000,7478.000000,7984.000000,7479.000000,7984.000000,4451.000000,8990.000000,8990.000000,8990.000000
mean,4494.500000,0.031591,0.031591,0.047831,0.575195,0.190656,-0.238042,0.045717,0.142825,0.143159,...,0.489553,0.506930,0.372494,0.287179,0.148270,0.303908,0.129227,0.000469,0.000107,0.000051
std,2595.333794,0.174917,0.174917,0.213420,0.494341,0.392840,0.425909,0.208883,0.349914,0.350254,...,0.306230,0.306554,1.153280,0.312238,1.326879,0.351109,1.277273,0.010551,0.000088,0.010568
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000661,0.000661,-2.723527,0.000661,-2.027551,0.000661,-1.497420,-0.039754,-0.000004,-0.040582
25%,2247.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.207011,0.236111,-0.472864,0.000661,-0.797168,0.000661,-0.738242,-0.004324,0.000008,-0.004759
50%,4494.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.492394,0.519180,0.275162,0.180225,-0.097670,0.101025,-0.170825,0.000659,0.000095,0.000252
75%,6741.750000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.760582,0.772487,1.125069,0.515873,0.795763,0.590030,0.685907,0.005900,0.000193,0.005479
max,8989.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,6.809912,1.000000,12.678264,1.000000,12.997536,0.040661,0.000317,0.040551


In [4]:
from python.metrics import score

In [ ]:
res = []

allocation_range = np.linspace(0, 2, 200)[1:] # 0-2の範囲で200分割
for allocation in allocation_range:
    # すべて同じallocationを割り当てた場合の最終スコアを計算
    submission = pd.DataFrame({'prediction': np.ones(len(train)) * allocation}, index=train.index)

    res.append(score(train.copy(), submission, ''))

In [ ]:
res # -> 0.8くらいが最適解っぽい

[0.2658337331658398,
 0.32098281626730557,
 0.33704811161631393,
 0.3446757668220962,
 0.34960510920671933,
 0.3534281710525361,
 0.3567169260279128,
 0.3597155231986711,
 0.3625416281271135,
 0.36525723102630914,
 0.3678972519666936,
 0.3704823842985125,
 0.3730253616802301,
 0.37553422904746975,
 0.3780141475640615,
 0.38046843832106575,
 0.38289921045758574,
 0.38530775250815713,
 0.38769478371554567,
 0.390060619836911,
 0.39240528517956985,
 0.3947285900086801,
 0.3970301851008336,
 0.3993096009211737,
 0.40156627626825153,
 0.4037995795710398,
 0.4060088249886938,
 0.40819328478814126,
 0.41035219901106684,
 0.41248478313720205,
 0.414590234290438,
 0.41666773631013504,
 0.41871646399180656,
 0.4207355866834555,
 0.42272427137861146,
 0.4246816854204554,
 0.4266069989110426,
 0.4284993868616416,
 0.4303580311607976,
 0.4321821223828665,
 0.433970861458284,
 0.43572346124409966,
 0.43743914799851835,
 0.4391171627732656,
 0.4407567627400652,
 0.44235722245477715,
 0.44391783505663